In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import cv2
import os
from keras.preprocessing import image
import numpy as np
from keras import layers, models, optimizers, applications
from tqdm import tqdm
import shutil
from keras.callbacks import ModelCheckpoint, EarlyStopping

model = applications.MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3), alpha = 0.5, pooling = None)

In [0]:
model.output

In [0]:
x = model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
predictions = layers.Dense(2, activation='softmax')(x)

In [0]:
final_model = models.Model(inputs = model.input, outputs = predictions)

In [0]:
final_model.summary()

In [0]:
final_model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
class_names = ['Bad', 'Good']
data_dir = '/content/drive/My Drive/Colab Notebooks/shiftLab/Studying/Data/Augmented'
train_dir = 'train'
val_dir = 'val'

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok = True)

In [0]:
for class_name in class_names:
    source_dir = os.path.join(data_dir, class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:  
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [0]:
train_datagen = image.ImageDataGenerator()
val_datagen = image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (224, 224), batch_size = 16, class_mode = 'categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size = (224, 224), class_mode = 'categorical',  batch_size = 16)

In [0]:
checkpoint = ModelCheckpoint("mbl.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=1, verbose=1, mode='auto')

In [0]:
final_model.fit_generator(
train_generator,
steps_per_epoch = 43,
verbose = 1,
epochs = 10,
validation_data = val_generator,
validation_steps = 35,
callbacks = [checkpoint, early])

In [0]:
img = image.load_img('/content/drive/My Drive/Colab Notebooks/shiftLab/Studying/Data/Dataset/Good/Old/53.jpg', target_size = (224, 224))
img = np.expand_dims(img, axis=0)
img.shape

In [0]:
final_model.predict(img)

In [0]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model_file('mbl.h5')
tfmodel = converter.convert()
open ("mbl.tflite" , "wb") .write(tfmodel)